# Medicare Part D Drug Spending Data <small>Downloading, extracting, and cleaning</small>

In [17]:
import requests # to download the dataset
import zipfile # to extract from archive
import shutil # to write the dataset to file
import os # rename file to something more type-able
import pandas as pd
import numpy as np

url = 'https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/Information-on-Prescription-Drugs/Downloads/Part_D_All_Drugs_2015.zip'
response = requests.get(url, stream=True)

zip_filename = 'dataset.zip'
with open(zip_filename, 'wb') as ds_zipout:
    shutil.copyfileobj(response.raw, ds_zipout)

zip = zipfile.ZipFile(zip_filename, 'r')
xls_filename = zip.namelist()[0]
zip.extract(xls_filename)

os.remove(zip_filename)

We're going to read the data into a `pandas.DataFrame`, but because the data is in several work sheets, we need to read out the right work sheet:

In [18]:
xls = pd.ExcelFile(xls_filename)
df = xls.parse('Data', skiprows=3)
df.index = np.arange(1, len(df) + 1)

Now we need to start capturing relevant data. The easiest way to start is by getting a list of prescribed drug names, both brand and generic.

In [19]:
# Capture ONLY the drug names (the first two columns)
df_drugnames = df.iloc[:, :2]
df_drugnames.columns = [
    'drugname_brand',
    'drugname_generic'
]

The drug names may contain leading or trailing whitespace in their "raw" state direct from the XLSX file; let's scrub that whitespace.

In [20]:
# Strip extraneous whitespace from drug names
df_drugnames.loc[:, 'drugname_brand'] = df_drugnames.loc[:, 'drugname_brand'].map(lambda x: x.strip())
df_drugnames.loc[:, 'drugname_generic'] = df_drugnames.loc[:, 'drugname_generic'].map(lambda x: x.strip())

Now that this portion of the data is clean, we can safely write it out to both feather and CSV files.

In [21]:
# Write to feather ...
import feather
feather.write_dataframe(df_drugnames, 'drugnames.feather')

# Now write to CSV ...
df_drugnames.to_csv('drugnames.csv', index=False)

Okay! With drug names, out of the way, we can concentrate on annual spending data.

We can parse out spending for each year by extracting their corresponding column ranges in the XLS. After we do that, it's easy to add back the drug names we just gathered. That way, people don't have to reference two DataFrames every time they want to track drug spending for a given year.

In [22]:
# Separate column groups by year
cols_by_year = [
    { 'year': 2011, 'start': 2, 'end': 12 },
    { 'year': 2012, 'start': 12, 'end': 22 },
    { 'year': 2013, 'start': 22, 'end': 32 },
    { 'year': 2014, 'start': 32, 'end': 42 },
    { 'year': 2015, 'start': 42, 'end': 53 },
]

df_years = {}

col_brandname = 0
col_genericname = 1
for cols in cols_by_year:
    year, start, end = cols['year'], cols['start'], cols['end']

    df_years[year] = pd.concat(
        [
            df_drugnames,
            df.iloc[:, start:end]
        ],
        axis=1)

Let's go ahead and remove the final column of the 2015 data ("Annual Change in Average Cost per Unit"). None of the other years have it, so it doesn't give us a lot of opportunity for comparison and analysis; plus, it's only available for drugs used at least 1,000 beneficiaries.

In [23]:
df_years[2015] = df_years[2015].drop(df_years[2015].columns[-1], axis=1)

From year to year, the list of prescribed drugs changes; some are added, and some are removed. We should probably get rid of any rows where that drug was not prescribed, just to reduce noise in our data.

In [24]:
for year in df_years:
    nonnull_rows = df_years[year].iloc[:, 2:].apply(lambda x: x.notnull().any(), axis=1)
    df_years[year] = df_years[year][nonnull_rows]
    df_years[year].index = np.arange(1, len(df_years[year]) + 1)

Let's make our column names easier to type and not year-specific, while still retaining most of their original meanings.

In [25]:
generic_columns = [
    "drugname_brand",
    "drugname_generic",
    "claim_count",
    "total_spending",
    "user_count",
    "total_spending_per_user",
    "unit_count",
    "unit_cost_wavg",
    "user_count_non_lowincome",
    "out_of_pocket_avg_non_lowincome",
    "user_count_lowincome",
    "out_of_pocket_avg_lowincome"
]

for year in df_years:
    df_years[year].columns = generic_columns

All fields in the annual spending data -- excluding drug names -- are numeric. Let's make sure pandas knows this, because otherwise we'll have trouble writing the data to feather and CSV.

In [26]:
# Suppress SettingWithCopyWarnings because I think it's
# tripping on the fact that we have a dict of DataFrames
pd.options.mode.chained_assignment = None
for year in df_years:
    # Ignore the first two columns, which are strings and contain drug names
    for col in df_years[year].columns[2:]:
        df_years[year].loc[:, col] = pd.to_numeric(df_years[year][col])
pd.options.mode.chained_assignment = 'warn'

All done cleaning the annual spending data! Let's write to disk in feather and CSV format

In [29]:
# Write each year's data
for year in df_years:
    # To feather ...
    feather.write_dataframe(df_years[year], 'spending-{year}.feather'.format(year=year))
    # And to CSV ...
    df_years[year].to_csv('spending-{year}.csv'.format(year=year), index=False)